
# Cofactors merging databases

This notebook merges the cofactor-protein interatomic interactions retrieved by PDB API and then map them to different databases, merging them and generating useful data tables.
 
**What this notebook does**  
- Loads your CSV files.
- Normalizes column names.
- Saves tables to CSV and shows previews.


In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

In [5]:
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)

DATA_DIR = Path(".")  # change if your CSVs live elsewhere

files = {
    "cofactors_atp": DATA_DIR / "results_mmciff__early_selab_1000_all_all_1_example.csv", 
    "cofactor_names": DATA_DIR / "cofactor_names_file.csv",
}

def _drop_unnamed(df: pd.DataFrame) -> pd.DataFrame:
    return df.loc[:, ~df.columns.str.contains(r'^Unnamed', case=False)]

def _normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out.columns = (
        out.columns
        .str.strip().str.replace(" ", "_", regex=False)
        .str.replace("-", "_", regex=False)
        .str.replace(".", "_", regex=False)
        .str.lower()
    )
    return out

def _to_str(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        try:
            out[c] = out[c].astype(str)
        except Exception:
            pass
    return out

def safe_read_csv(path: Path, **kwargs) -> pd.DataFrame:
    df = pd.read_csv(path, **kwargs)
    return _to_str(_normalize_columns(_drop_unnamed(df)))

missing = [k for k,p in files.items() if not p.exists()]
if missing:
    print("[ERROR] Missing files:", {k: str(files[k]) for k in missing})
    print("Place the files or update `DATA_DIR` / filenames above, then re-run.")
else:
    print("[OK] Found all files.")


[OK] Found all files.


In [6]:

# Load (or stop with clear message)
if all(p.exists() for p in files.values()):
    cofactors_atp = safe_read_csv(files["cofactors_atp"])
    cofactor_names = safe_read_csv(files["cofactor_names"])
    print("cofactors_atp shape:", cofactors_atp.shape)
    print("cofactor_names shape:", cofactor_names.shape)
    print("\ncofactors_atp columns:", sorted(cofactors_atp.columns.tolist())[:40], "...")
    print("cofactor_names columns:", sorted(cofactor_names.columns.tolist())[:40], "...")
else:
    raise SystemExit("Fix missing files and re-run.")


cofactors_atp shape: (5999, 17)
cofactor_names shape: (467, 2)

cofactors_atp columns: ['amino_acid', 'aminoacid_type', 'atom_names_features', 'auth_res_num_request', 'chain_atom_type', 'chain_id', 'chain_request', 'chem_comp_id_api', 'distance_a', 'interaction_details', 'interaction_type', 'ligand_atoms', 'pdb_id', 'pdbe_residue', 'sec_structure', 'sequence_residue', 'uniprot'] ...
cofactor_names columns: ['chem_comp_id_api', 'class'] ...



## Key discovery and ranking

In [8]:

# Candidate keys: shared columns + plausible synonyms
shared = sorted(set(cofactors_atp.columns) & set(cofactor_names.columns))

# Add extra plausible keys if present under different names in one side
synonyms = [
    ("chem_comp_id_api", "chem_comp_id_api"),
    ("chem_comp_id", "chem_comp_id"),
    ("het_id", "het_id"),
    ("ligand_id", "ligand_id"),
    ("res_name", "res_name"),
    ("aminoacid_type", "aminoacid_type"),
    ("residue", "residue"),
]
for a, b in synonyms:
    if a in cofactors_atp.columns and b in cofactor_names.columns and a not in shared:
        shared.append(a)

def key_score(left: pd.DataFrame, right: pd.DataFrame, key: str) -> dict:
    L = left[key].dropna().astype(str)
    R = right[key].dropna().astype(str)
    overlap = len(set(L) & set(R))
    right_unique = not right.duplicated(subset=[key]).any()
    # try small sample inner join to see if matches exist
    sample_left = left[[key]].drop_duplicates().head(20000)
    inner = sample_left.merge(right[[key]].drop_duplicates(), on=key, how="inner")
    return {
        "key": key,
        "overlap": overlap,
        "right_unique": right_unique,
        "inner_has_matches": inner.shape[0] > 0
    }

scores = [key_score(cofactors_atp, cofactor_names, k) for k in shared]
scores = sorted(scores, key=lambda d: (d["inner_has_matches"], d["right_unique"], d["overlap"]), reverse=True)

if not scores:
    raise SystemExit("No candidate join keys found. Inspect column lists above and add a mapping.")

pd.DataFrame(scores).head(10)


,key,overlap,right_unique,inner_has_matches
0,chem_comp_id_api,9,True,True


## Merge execution

In [10]:

merged = None
used_key = None
method = None

for cand in scores:
    k = cand["key"]
    if not cand["inner_has_matches"]:
        continue

    # if RHS unique, do a clean many_to_one
    if cand["right_unique"]:
        try:
            merged_try = cofactors_atp.merge(
                cofactor_names.drop_duplicates(subset=[k]),
                how="left",
                on=k,
                validate="many_to_one",
            )
            if merged_try.shape[0] > 0:
                merged, used_key, method = merged_try, k, "many_to_one"
                break
        except Exception as e:
            print(f"[WARN] many_to_one failed on key '{k}': {e}")

    # else aggregate RHS to make it unique
    # pick all non-key columns to aggregate with ';' joining unique non-null strings
    rhs = cofactor_names.copy()
    rhs_cols = [c for c in rhs.columns if c != k]
    agg_map = {c: (lambda s: ";".join(sorted({str(x) for x in s if pd.notna(x)}))) for c in rhs_cols}

    try:
        rhs_agg = rhs.groupby(k, as_index=False).agg(agg_map)
        merged_try = cofactors_atp.merge(rhs_agg, how="left", on=k, validate="many_to_one")
        if merged_try.shape[0] > 0:
            merged, used_key, method = merged_try, k, "aggregated_right"
            break
    except Exception as e:
        print(f"[WARN] aggregation merge failed on key '{k}': {e}")

if merged is None:
    raise SystemExit("No candidate key produced a non-empty merge. "
                     "Check the column audit above; you may need to specify the correct key explicitly.")
else:
    print(f"[OK] Merged on key '{used_key}' via method '{method}' -> shape {merged.shape}")
    display(merged.head(3))


[OK] Merged on key 'chem_comp_id_api' via method 'many_to_one' -> shape (5999, 18)


,ligand_atoms,interaction_type,interaction_details,amino_acid,chain_id,sequence_residue,atom_names_features,distance_a,pdb_id,chain_request,auth_res_num_request,chem_comp_id_api,chain_atom_type,aminoacid_type,uniprot,pdbe_residue,sec_structure,class
0,SD,atom-atom,hydrophobic,DC,D,427,C5,3.97,2c7p,A_1,1328,SAH,side_chain,other,missing,6,nan,S-adenosylmethionine
1,SD,atom-atom,hydrophobic,DC,D,427,C5,3.96,2uyh,A_1,1328,SAH,side_chain,other,missing,6,nan,S-adenosylmethionine
2,CE,atom-atom,vdw_clash,DA,D,6,C6,3.38,2y7h,B_1,530,SAM,side_chain,other,missing,6,nan,S-adenosylmethionine


In [11]:
# 1) Load EC table if not loaded already
try:
    ec_df
except NameError:
    import pandas as pd
    from pathlib import Path
    for fn in ["pdb_chain_enzyme_df_c.csv", "pdb_chain_enzyme.csv"]:
        if Path(fn).exists():
            ec_df = pd.read_csv(fn)
            break
    else:
        raise FileNotFoundError("EC file not found. Place 'pdb_chain_enzyme_df_c.csv' next to this notebook.")

    ec_df.columns = (
        ec_df.columns
        .str.strip().str.lower()
        .str.replace(" ", "_", regex=False)
        .str.replace("-", "_", regex=False)
        .str.replace(".", "_", regex=False)
    )
    ec_df = ec_df.rename(columns={
        "pdb": "pdb_id",
        "pdb_code": "pdb_id",
        "chainpdb": "chain_id",
        "chain": "chain_id",
        "chainid": "chain_id",
        "ec_number": "ec",
        "ecnum": "ec",
        "uniprot_id": "uniprot",
        "uniprot ac": "uniprot",
    })

# 2) Drop duplicate ECs and merge
if "ec" in merged.columns:
    merged = merged.drop(columns=["ec"])

if {"pdb_id","chain_id","ec"}.issubset(ec_df.columns) and {"pdb_id","chain_id"}.issubset(merged.columns):
    ec_rhs = ec_df[["pdb_id","chain_id","ec"]].drop_duplicates(["pdb_id","chain_id"])
    merged = merged.merge(ec_rhs, how="left", on=["pdb_id","chain_id"])
elif {"pdb_id","ec"}.issubset(ec_df.columns) and "pdb_id" in merged.columns:
    ec_rhs = ec_df[["pdb_id","ec"]].drop_duplicates("pdb_id")
    merged = merged.merge(ec_rhs, how="left", on="pdb_id")
elif {"uniprot","ec"}.issubset(ec_df.columns) and "uniprot" in merged.columns:
    ec_rhs = ec_df[["uniprot","ec"]].drop_duplicates("uniprot")
    merged = merged.merge(ec_rhs, how="left", on="uniprot")
else:
    print("EC merge skipped: no overlapping keys found.")

# 3) Save the completed merged CSV
merged.to_csv("cofactors_atp__Cofactorsname_df__all_domains_complete__EC_df.csv", index=False)
print("Saved: cofactors_atp__Cofactorsname_df__all_domains_complete__EC_df.csv",
      "| Rows with EC:", merged['ec'].notna().sum() if 'ec' in merged.columns else 0)
display(merged.head(3))


Saved: cofactors_atp__Cofactorsname_df__all_domains_complete__EC_df.csv | Rows with EC: 0


,ligand_atoms,interaction_type,interaction_details,amino_acid,chain_id,sequence_residue,atom_names_features,distance_a,pdb_id,chain_request,auth_res_num_request,chem_comp_id_api,chain_atom_type,aminoacid_type,uniprot,pdbe_residue,sec_structure,class,ec
0,SD,atom-atom,hydrophobic,DC,D,427,C5,3.97,2c7p,A_1,1328,SAH,side_chain,other,missing,6,nan,S-adenosylmethionine,NaN
1,SD,atom-atom,hydrophobic,DC,D,427,C5,3.96,2uyh,A_1,1328,SAH,side_chain,other,missing,6,nan,S-adenosylmethionine,NaN
2,CE,atom-atom,vdw_clash,DA,D,6,C6,3.38,2y7h,B_1,530,SAM,side_chain,other,missing,6,nan,S-adenosylmethionine,NaN



## Build tables

In [13]:

# We want: category (like 'class') and type (like 'aminoacid_type')
# Provide synonyms to auto-discover columns.
CAND_CLASS = ["class", "group", "category", "set", "label"]
CAND_TYPE  = ["aminoacid_type", "amino_acid_type", "aa_type", "type", "family"]

def pick_first_existing(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

col_class = pick_first_existing(merged, CAND_CLASS)
col_type  = pick_first_existing(merged, CAND_TYPE)

print("Chosen columns -> class:", col_class, "| type:", col_type)

if col_class is None or col_type is None:
    available = merged.columns.tolist()
    raise SystemExit("Cannot build tables because required columns are missing. "
                     f"Looked for class in {CAND_CLASS} and type in {CAND_TYPE}. "
                     f"Available columns: {available[:60]} ...")

# Counts
table_counts = pd.crosstab(
    index=merged[col_class],
    columns=merged[col_type],
    margins=True,
    margins_name="All"
)
table_counts.to_csv("table_counts_auto.csv")
print("[OK] Wrote table_counts_auto.csv")
display(table_counts.head())

# Row percentages
table_perc = pd.crosstab(
    index=merged[col_class],
    columns=merged[col_type],
    normalize="index"
) * 100.0
table_perc["All"] = 100.0
table_perc.to_csv("table_percent_auto.csv")
print("[OK] Wrote table_percent_auto.csv")
try:
    display(table_perc.style.format("{:.1f}%").set_caption("Row %"))
except Exception:
    display(table_perc.head())
 

Chosen columns -> class: class | type: aminoacid_type
[OK] Wrote table_counts_auto.csv


aminoacid_type,early_AA,late_AA,other,All
class,,,,
ATP_ligand,229,505,5152,5886
Coenzyme_A,0,0,27,27
Flavin_adenine_dinucleotide,0,0,14,14
Nicotinamide-adenine_dinucleotide,0,0,22,22
S-adenosylmethionine,0,6,44,50


[OK] Wrote table_percent_auto.csv


aminoacid_type,early_AA,late_AA,other,All
class,,,,
ATP_ligand,3.9%,8.6%,87.5%,100.0%
Coenzyme_A,0.0%,0.0%,100.0%,100.0%
Flavin_adenine_dinucleotide,0.0%,0.0%,100.0%,100.0%
Nicotinamide-adenine_dinucleotide,0.0%,0.0%,100.0%,100.0%
S-adenosylmethionine,0.0%,12.0%,88.0%,100.0%



## Optional: two‑level (class, amino_acid) table


In [15]:

# Try to find an amino_acid column
CAND_AA = ["amino_acid", "aa", "residue", "res_name"]
col_aa = pick_first_existing(merged, CAND_AA)

if col_aa:
    table_aa = pd.crosstab(
        index=[merged[col_class], merged[col_aa]],
        columns=merged[col_type],
        margins=True,
        margins_name="All"
    )
    table_aa.to_csv("table_class_aa_auto.csv")
    print("[OK] Wrote table_class_aa_auto.csv")
    display(table_aa.head())
else:
    print("[Info] No amino acid column found among", CAND_AA, "- skipping two-level table.")


[OK] Wrote table_class_aa_auto.csv


aminoacid_type         early_AA  late_AA  other   All
class      amino_acid                                
ATP_ligand ARG                0       18      0    18
           ASN                0       20      0    20
           ASP               38        0      0    38
           DA                 0        0   1074  1074
           DC                 0        0   1173  1173